<a href="https://colab.research.google.com/github/jeaneigsi/Humain-/blob/main/instruct_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/kevinscaria/InstructABSA/tree/main   
https://huggingface.co/yangheng/deberta-v3-large-absa-v1.1   
wget https://github.com/kevinscaria/InstructABSA/blob/main/instructions.py

In [60]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-large-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

for aspect in word_list :
   print(aspect, classifier('the screen is bad and phone also but design is amazing',  text_pair=aspect))


camera [{'label': 'Negative', 'score': 0.997731626033783}]
quality, [{'label': 'Negative', 'score': 0.9094060659408569}]
battery [{'label': 'Negative', 'score': 0.5619088411331177}]
life, [{'label': 'Positive', 'score': 0.9305965900421143}]
screen [{'label': 'Negative', 'score': 0.9998279809951782}]
brightness, [{'label': 'Negative', 'score': 0.7214969396591187}]
design, [{'label': 'Positive', 'score': 0.9998764991760254}]
weight, [{'label': 'Positive', 'score': 0.9953311085700989}]
phone [{'label': 'Negative', 'score': 0.9997301697731018}]


In [56]:
word_list

['camera',
 'quality,',
 'battery',
 'life,',
 'screen',
 'brightness,',
 'design,',
 'weight,',
 'phone']

warnings.warn(
camera [{'label': 'Positive', 'score': 0.9967294931411743}]
phone [{'label': 'Neutral', 'score': 0.9472787380218506}]

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-large-absa-v1.1"
tokenizers = AutoTokenizer.from_pretrained(model_name)
models = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to run ABSA inference manually
def classify_text(model, tokenizer, text, aspect):
    # Prepare input as a pair of sentences (main text and aspect)
    inputs = tokenizers(text, aspect, return_tensors="pt", padding=True, truncation=True)

    # Forward pass through the model to get logits
    with torch.no_grad():
        outputs = models(**inputs)

    # Apply softmax to get probabilities
    probabilities = torch.softmax(outputs.logits, dim=-1)

    # Get the predicted class (argmax of the logits)
    predicted_class = torch.argmax(probabilities, dim=-1).item()

    # Map predicted class to sentiment labels (assumed 0=negative, 1=neutral, 2=positive)
    sentiment_labels = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiment_labels[predicted_class]

    return predicted_sentiment, probabilities

# Example text and aspects
text = "The camera quality of this phone is amazing."
aspects = ['camera', 'phone']

# Loop over aspects and classify
for aspect in aspects:
    sentiment, probabilities = classify_text(model, tokenizer, text, aspect)

    # Output similar to pipeline
    print(f"Aspect: {aspect}, Sentiment: {sentiment}, Probabilities: {torch.argmax(probabilities, dim=-1)}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Aspect: camera, Sentiment: positive, Probabilities: tensor([2])
Aspect: phone, Sentiment: neutral, Probabilities: tensor([1])


In [21]:
input="The food was delicious"
aspect="Service"
inputs = tokenizers(input, aspect, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = models(**inputs)
    pred=torch.argmax(outputs.logits, dim=-1)

sentiment=['negative','neutral','positive']
print(f"Sentiment : {sentiment[pred.item()]} and {torch.softmax(outputs.logits, dim=-1)}")

Sentiment : neutral and tensor([[4.9511e-04, 9.8943e-01, 1.0076e-02]])


# Instructabsa

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from transformers import (
    DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Trainer, Seq2SeqTrainer
)


class T5Generator:
    def __init__(self, model_checkpoint):
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
        self.data_collator = DataCollatorForSeq2Seq(self.tokenizer)
        self.device = 'cuda' if torch.has_cuda else ('mps' if torch.has_mps else 'cpu')

    def tokenize_function_inputs(self, sample):
        """
        Udf to tokenize the input dataset.
        """
        model_inputs = self.tokenizer(sample['text'], max_length=512, truncation=True)
        labels = self.tokenizer(sample["labels"], max_length=64, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    def train(self, tokenized_datasets, **kwargs):
        """
        Train the generative model.
        """
        #Set training arguments
        args = Seq2SeqTrainingArguments(
            **kwargs
        )

        # Define trainer object
        trainer = Seq2SeqTrainer(
            self.model,
            args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"] if tokenized_datasets.get("validation") is not None else None,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
        )
        print("Trainer device:", trainer.args.device)

        # Finetune the model
        torch.cuda.empty_cache()
        print('\nModel training started ....')
        trainer.train()

        # Save best model
        trainer.save_model()
        return trainer

    def get_labels(self, tokenized_dataset, batch_size = 4, max_length = 128, sample_set = 'train'):
        """
        Get the predictions from the trained model.
        """
        def collate_fn(batch):
            input_ids = [torch.tensor(example['input_ids']) for example in batch]
            input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            return input_ids

        dataloader = DataLoader(tokenized_dataset[sample_set], batch_size=batch_size, collate_fn=collate_fn)
        predicted_output = []
        self.model.to(self.device)
        print('Model loaded to: ', self.device)

        for batch in tqdm(dataloader):
            batch = batch.to(self.device)
            output_ids = self.model.generate(batch, max_length = max_length)
            output_texts = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for output_text in output_texts:
                predicted_output.append(output_text)
        return predicted_output

    def get_metrics(self, y_true, y_pred, is_triplet_extraction=False):
        total_pred = 0
        total_gt = 0
        tp = 0
        if not is_triplet_extraction:
            for gt, pred in zip(y_true, y_pred):
                gt_list = gt.split(', ')
                pred_list = pred.split(', ')
                total_pred+=len(pred_list)
                total_gt+=len(gt_list)
                for gt_val in gt_list:
                    for pred_val in pred_list:
                        if pred_val in gt_val or gt_val in pred_val:
                            tp+=1
                            break

        else:
            for gt, pred in zip(y_true, y_pred):
                gt_list = gt.split(', ')
                pred_list = pred.split(', ')
                total_pred+=len(pred_list)
                total_gt+=len(gt_list)
                for gt_val in gt_list:
                    gt_asp = gt_val.split(':')[0]

                    try:
                        gt_op = gt_val.split(':')[1]
                    except:
                        continue

                    try:
                        gt_sent = gt_val.split(':')[2]
                    except:
                        continue

                    for pred_val in pred_list:
                        pr_asp = pred_val.split(':')[0]

                        try:
                            pr_op = pred_val.split(':')[1]
                        except:
                            continue

                        try:
                            pr_sent = gt_val.split(':')[2]
                        except:
                            continue

                        if pr_asp in gt_asp and pr_op in gt_op and gt_sent == pr_sent:
                            tp+=1

        p = tp/total_pred
        r = tp/total_gt
        return p, r, 2*p*r/(p+r), None


class T5Classifier:
    def __init__(self, model_checkpoint):
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, force_download = True)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, force_download = True)
        self.data_collator = DataCollatorForSeq2Seq(self.tokenizer)
        self.device = 'cuda' if torch.has_cuda else ('mps' if torch.has_mps else 'cpu')

    def tokenize_function_inputs(self, sample):
        """
        Udf to tokenize the input dataset.
        """
        sample['input_ids'] = self.tokenizer(sample["text"], max_length = 512, truncation = True).input_ids
        sample['labels'] = self.tokenizer(sample["labels"], max_length = 64, truncation = True).input_ids
        return sample

    def train(self, tokenized_datasets, **kwargs):
        """
        Train the generative model.
        """

        # Set training arguments
        args = Seq2SeqTrainingArguments(
            **kwargs
            )

        # Define trainer object
        trainer = Trainer(
            self.model,
            args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"] if tokenized_datasets.get("validation") is not None else None,
            tokenizer=self.tokenizer,
            data_collator = self.data_collator
        )
        print("Trainer device:", trainer.args.device)

        # Finetune the model
        torch.cuda.empty_cache()
        print('\nModel training started ....')
        trainer.train()

        # Save best model
        trainer.save_model()
        return trainer

    def get_labels(self, tokenized_dataset, batch_size = 4, sample_set = 'train'):
        """
        Get the predictions from the trained model.
        """
        def collate_fn(batch):
            input_ids = [torch.tensor(example['input_ids']) for example in batch]
            input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            return input_ids

        dataloader = DataLoader(tokenized_dataset[sample_set], batch_size=batch_size, collate_fn=collate_fn)
        predicted_output = []
        self.model.to(self.device)
        print('Model loaded to: ', self.device)

        for batch in tqdm(dataloader):
            batch = batch.to(self.device)
            output_ids = self.model.generate(batch)
            output_texts = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for output_text in output_texts:
                predicted_output.append(output_text)
        return predicted_output

    def get_metrics(self, y_true, y_pred):
        return precision_score(y_true, y_pred, average='macro'), recall_score(y_true, y_pred, average='macro'), \
            f1_score(y_true, y_pred, average='macro'), accuracy_score(y_true, y_pred)

In [ ]:
kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined

In [ ]:
t5_exp = T5Generator(model_checkpoint)
bos_instruction_id = instruct_handler.ate[indomain]
if ood_tr_data_path is not None or ood_te_data_path is not None:
    bos_instruction_ood = instruct_handler.ate[outdomain]
eos_instruction = instruct_handler.ate['eos_instruct']

In [ ]:
!pip install sentencepiece evaluate datasets

In [24]:
!git clone https://huggingface.co/yangheng/deberta-v3-large-absa-v1.1

Cloning into 'deberta-v3-large-absa-v1.1'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 30 (delta 5), reused 2 (delta 2), pack-reused 18 (from 1)
Unpacking objects: 100% (30/30), 8.42 KiB | 1.05 MiB/s, done.
Filtering content: 100% (3/3), 3.24 GiB | 43.93 MiB/s, done.


In [37]:
from instructions import InstructionsHandler
instruct_handler = InstructionsHandler()
instruct_handler.load_instruction_set1()
bos_instruction_id = instruct_handler.ate['bos_instruct1']
bos_instruction_id


'Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.\n        Positive example 1-\n        input: I charge it at night and skip taking the cord with me because of the good battery life.\n        output: battery life\n        Positive example 2-\n        input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.\n        output: features, iChat, Photobooth, garage band\n        Now complete the following example-\n        input: '

In [48]:
test_input="I recently purchased the new XYZ smartphone, and overall, I'm quite impressed. The camera quality is exceptional, capturing vibrant and clear photos even in low-light conditions. The battery life is also impressive; I can easily get through a full day of heavy use without needing to recharge. However, the screen brightness could be better—it's a bit difficult to see under direct sunlight. Additionally, while the design is sleek and modern, the weight of the phone feels a bit cumbersome after prolonged use. The customer service experience was positive, with prompt responses and helpful support."

In [41]:
import torch
# from InstructABSA.utils import T5Generator
# from InstructABSA.config import Config
from instructions import InstructionsHandler

model_checkpoint="kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"

instruct_handler = InstructionsHandler()
instruct_handler.load_instruction_set1()

# Load the T5 model for ATE task
t5_exp = T5Generator(model_checkpoint)

# Load ATE instructions
bos_instruction_id = instruct_handler.ate['bos_instruct1']
eos_instruction = instruct_handler.ate['eos_instruct']

# Single input for inference
#test_input = "The camera quality of this phone is amazing."

# Prepare the model input
model_input = bos_instruction_id + test_input + eos_instruction

# Tokenize input
input_ids = t5_exp.tokenizer(model_input, return_tensors="pt").input_ids




/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-22-71b899bf3f3f>:18: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  self.device = 'cuda' if torch.has_cuda else ('mps' if torch.has_mps else 'cpu')


In [46]:
# Perform inference
with torch.no_grad():
    outputs = t5_exp.model.generate(input_ids, max_length=1024)

# Decode and print the output
predicted_text = t5_exp.tokenizer.decode(outputs[0], skip_special_tokens=True)
word_list=predicted_text.split()
print(f"Model output: {predicted_text}")

Model output: camera quality, battery life, screen brightness, design, weight, phone


In [45]:
outputs[0]

tensor([    0,  1861,   463,     6,  3322,   280,     6,  1641, 24073,     6,
          408,     6,  1293,     6,   951,     1])

In [54]:
word_list

['camera',
 'quality,',
 'battery',
 'life,',
 'screen',
 'brightness,',
 'design,',
 'weight,',
 'phone']

In [49]:
aspects=word_list

In [53]:
text = "i love camera"
#aspects = ['camera', 'phone']

# Loop over aspects and classify
for aspect in aspects:
    sentiment, probabilities = classify_text(model, tokenizer, text, aspect)

    # Output similar to pipeline
    print(f"Aspect: {aspect}, Sentiment: {sentiment}, Probabilities: {torch.argmax(probabilities, dim=-1)}")

Aspect: camera, Sentiment: positive, Probabilities: tensor([2])
Aspect: quality,, Sentiment: positive, Probabilities: tensor([2])
Aspect: battery, Sentiment: positive, Probabilities: tensor([2])
Aspect: life,, Sentiment: positive, Probabilities: tensor([2])
Aspect: screen, Sentiment: positive, Probabilities: tensor([2])
Aspect: brightness,, Sentiment: positive, Probabilities: tensor([2])
Aspect: design,, Sentiment: positive, Probabilities: tensor([2])
Aspect: weight,, Sentiment: positive, Probabilities: tensor([2])
Aspect: phone, Sentiment: positive, Probabilities: tensor([2])
